In [ ]:
# Configuración básica del proyecto
PROJECT_NAME = 'Bayesian NN'

import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from classes.DataHandler import DataHandler
from classes.BayesianNN_opt import BayesianNN_opt

dh = DataHandler()
DEVICE = 'cuda'
dh.DEVICE = DEVICE

dh.set_targets(['Mo_cumulative'])
table_unified, vars_desc = dh.load_data()
inputs, targets = dh.preprocess_data(vars_desc, table_unified)
X, y, scales = dh.normalize_data(inputs, targets)

# Convertir X y y a tensores de PyTorch y moverlos a la GPU
X = torch.tensor(X, dtype=torch.float32).to(dh.DEVICE)
y = torch.tensor(y, dtype=torch.float32).to(dh.DEVICE)

# Dividir los datos en entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(
    X.cpu().numpy(), y.cpu().numpy(), test_size=0.2, random_state=42
)

# Convertir los conjuntos a tensores y moverlos a la GPU
X_train = torch.tensor(X_train, dtype=torch.float32).to(dh.DEVICE)
X_val = torch.tensor(X_val, dtype=torch.float32).to(dh.DEVICE)
y_train = torch.tensor(y_train, dtype=torch.float32).to(dh.DEVICE)
y_val = torch.tensor(y_val, dtype=torch.float32).to(dh.DEVICE)

input_dim = X_train.shape[1]
hidden_dim = 512
show_info = 'True'

bnn = BayesianNN_opt(input_dim, hidden_dim, DEVICE, show_info=show_info)
bnn.train_model(X_train, y_train, num_steps=10000, lr=1e-3)

In [ ]:
# Predicción y desnormalización
y_pred_train, y_pred_train_std = bnn.predict(X_train, num_samples=100)
y_pred_val, y_pred_val_std = bnn.predict(X_val, num_samples=100)
y_train_np = y_train.cpu().numpy()
y_val_np = y_val.cpu().numpy()
y_train_mo = dh.denormalize_value(scales, y_train_np, 'Mo_cumulative')
y_pred_train_mo = dh.denormalize_value(scales, y_pred_train, 'Mo_cumulative')
y_val_mo = dh.denormalize_value(scales, y_val_np, 'Mo_cumulative')
y_pred_val_mo = dh.denormalize_value(scales, y_pred_val, 'Mo_cumulative')

In [ ]:
# Graficar resultados con intervalo de confianza
dh.plot_with_confidence_interval(
    (y_train_mo, 'Valores Reales'),
    (y_pred_train_mo, y_pred_train_mo - 1.96 * y_pred_train_std, y_pred_train_mo + 1.96 * y_pred_train_std, 'BNN - Predicciones'),
    title='BNN: Predicciones con Intervalo (Entrenamiento)',
    size=(1200, 400)
)
dh.plot_with_confidence_interval(
    (y_val_mo, 'Valores Reales'),
    (y_pred_val_mo, y_pred_val_mo - 1.96 * y_pred_val_std, y_pred_val_mo + 1.96 * y_pred_val_std, 'BNN - Predicciones'),
    title='BNN: Predicciones con Intervalo (Validación)',
    size=(1200, 400)
)

In [ ]:
# Evaluación
dh.evaluar_metricas(y_train_mo, y_pred_train_mo, 'BNN - Entrenamiento')
dh.evaluar_metricas(y_val_mo, y_pred_val_mo, 'BNN - Validación')

In [ ]:
# Gráficos de regresión
dh.plot_regression(y_train_mo, y_pred_train_mo, title='Regresión BNN - Entrenamiento', size=(1200, 400))
dh.plot_regression(y_val_mo, y_pred_val_mo, title='Regresión BNN - Validación', size=(1200, 400))